# Political TV Ads Initial EDA | Cleaning | Feature Engineering

In [1]:
import pprint
import pickle
import csv
import pandas as pd
from datetime import datetime
import numpy as np
import string
from collections import Counter
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl

pd.set_option("display.max_columns", 2000)
pd.set_option("display.max_rows", 2000)
# pd.set_option("display.precision", 2)
pd.options.display.float_format = '{:,.2f}'.format # this is superior to precision - b/c no scientific notation

In [2]:
# initial reading in of dataset -- large dataset -- takes time to load
# 'parse_dates' parameter - read in dates as datetime objects
# much more time more efficient to read in pickled dataset instead of csv

# all_df = pd.read_csv('data/political_ads.csv', parse_dates=['start_time', 'end_time', 'date_created'])

In [3]:
# all_df.head(2)

In [4]:
# saving as a pickle object to retain datetime object

# all_df.to_pickle('data/all_political_ads.pickle')

In [5]:
# reading in pickled dataset - more time efficient than reading in csv and parsing in dates as datetime objects

all_df = pd.read_pickle('data/all_political_ads.pickle')
all_df.head(2)

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created
0,1,5643,KLAS,"Las Vegas, NV",8 News Now at 5 PM,news,2016-09-09 00:12:59,2016-09-09 00:13:29,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32
1,2,5643,WNCN,"Raleigh-Durham-Fayetteville, NC",North Carolina News at 500PM,news,2016-09-06 21:58:25,2016-09-06 21:58:55,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32


### Initial Exploration of Data

In [6]:
all_df.shape

(364718, 19)

In [7]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364718 entries, 0 to 364717
Data columns (total 19 columns):
id               364718 non-null int64
wp_identifier    364718 non-null int64
network          364718 non-null object
location         364693 non-null object
program          364704 non-null object
program_type     364718 non-null object
start_time       364718 non-null datetime64[ns]
end_time         364718 non-null datetime64[ns]
archive_id       364718 non-null object
embed_url        364718 non-null object
sponsors         363556 non-null object
sponsor_types    363556 non-null object
race             334846 non-null object
cycle            335617 non-null float64
subjects         343932 non-null object
candidates       339591 non-null object
type             364718 non-null object
message          364718 non-null object
date_created     364718 non-null datetime64[ns]
dtypes: datetime64[ns](3), float64(1), int64(2), object(13)
memory usage: 52.9+ MB


In [8]:
all_df.describe()

,id,wp_identifier,cycle
count,"364,718.00","364,718.00","335,617.00"
mean,"237,070.53","4,078.86","2,016.00"
std,"139,393.65","2,709.65",0.00
min,1.00,232.00,"2,016.00"
25%,"114,419.25","1,213.00","2,016.00"
50%,"222,862.50","4,685.00","2,016.00"
75%,"372,456.75","6,714.00","2,016.00"
max,"464,741.00","8,404.00","2,016.00"


In [9]:
all_df.columns

Index(['id', 'wp_identifier', 'network', 'location', 'program', 'program_type',
       'start_time', 'end_time', 'archive_id', 'embed_url', 'sponsors',
       'sponsor_types', 'race', 'cycle', 'subjects', 'candidates', 'type',
       'message', 'date_created'],
      dtype='object')

In [10]:
# number of unique items per feature

for column in all_df.columns:
    print ("{:<20s} {:>6.0f}".format(column, all_df[column].nunique()))
    
# review how to print to get the numbers lined up nicely-- see Course-Syllabus 9.6

id                   364718
wp_identifier          1304
network                 130
location                 25
program                3124
program_type              2
start_time           352660
end_time             352938
archive_id             1304
embed_url              1304
sponsors                197
sponsor_types            11
race                     28
cycle                     1
subjects                916
candidates              149
type                      2
message                   5
date_created          17705


**- from unique values, looks like database has ads for all races not just the presidential race**

In [11]:
unique_categories = ['network',
                     'location',
                     'program_type',
                     'sponsor_types',
                     'race',
                     'candidates',
                     'type',
                     'message'
                    ]

In [12]:
# seeing what the unique categories are within each column

for category in unique_categories:
    print (category, '\n')
    print (all_df[category].unique())
    print ('\n')

network 

['KLAS' 'WNCN' 'WFLA' 'KSNV' 'WTVT' 'WTOG' 'WLFL' 'WOIO' 'WRAL' 'WEWS'
 'WRAZ' 'KVVU' 'KCRG' 'WFTS' 'KFXA' 'KGAN' 'KTNV' 'WTXF' 'WJW' 'WKYC'
 'WCAU' 'KYW' 'WPVI' 'KUSA' 'WTMJ' 'KNTV' 'KOFY' 'CNNW' 'MSNBCW' 'KPNX'
 'WHDH' 'FOXNEWSW' 'CSPAN' 'WMUR' 'WBZ' 'WFXT' 'WCVB' 'KQED' 'FBC' 'KMTP'
 'KRON' 'KTVU' 'KSAZ' 'WISN' 'KGO' 'KNXV' 'KPHO' 'LINKTV' 'WITI' 'WDJT'
 'CNBC' 'KMGH' 'KCNC' 'WUSA' 'KDVR' 'WTTG' 'CSPAN2' 'CSPAN3' 'WJLA'
 'BLOOMBERG' 'KCSM' 'KPIX' 'WKMG' 'ALJAZAM' 'KCAU' 'COM' 'WABC' 'KDTV'
 'WNYW' 'KSTS' 'WRC' 'KTSF' 'WNBC' 'KOLO' 'WUVP' 'WCBS' 'KMEG' 'KTVN'
 'WKRC' 'WHO' 'KRNV' 'WVEC' 'WOI' 'WSPA' 'WXIX' 'WCPO' 'WLWT' 'WAVY' 'WVBT'
 'WTKR' 'WHNS' 'WYFF' 'KTIV' 'KWWL' 'KCCI' 'WLOS' 'KRXI' 'KDSM' 'KPTH'
 'KRDO' 'WPLG' 'WCNC' 'WTVJ' 'FRANCE24' 'WSVN' 'WBTV' 'WTSP' 'WIS' 'WFTV'
 'WLTX' 'WTVD' 'WOLO' 'WSET' 'WOFL' 'WSOC' 'WFOR' 'WSLS' 'WACH' 'WESH'
 'WJZY' 'WDBJ' 'WFXR' 'KKTV' 'KOAA' 'KXRM' 'KQEH' 'BBCAMERICA' 'KCNS'
 'BETW' 'BBCNEWS']


location 

['Las Vegas, NV' 'Raleigh-Du

### Coding Plan I

- limit ads only to presidential race
- focus on general election -- limit dates from after the conventions (8/1/16) to Election Day (11/8/16)

In [13]:
# using a copy to be safe

filter_df = all_df.copy()
filter_df.head(2)

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created
0,1,5643,KLAS,"Las Vegas, NV",8 News Now at 5 PM,news,2016-09-09 00:12:59,2016-09-09 00:13:29,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32
1,2,5643,WNCN,"Raleigh-Durham-Fayetteville, NC",North Carolina News at 500PM,news,2016-09-06 21:58:25,2016-09-06 21:58:55,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32


In [14]:
# filtering out all all dates before 8/1/2016

filter_df = filter_df[filter_df['start_time'] >= datetime(2016, 8, 1, 0, 0)]
filter_df.head(2)

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created
0,1,5643,KLAS,"Las Vegas, NV",8 News Now at 5 PM,news,2016-09-09 00:12:59,2016-09-09 00:13:29,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32
1,2,5643,WNCN,"Raleigh-Durham-Fayetteville, NC",North Carolina News at 500PM,news,2016-09-06 21:58:25,2016-09-06 21:58:55,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32


In [15]:
filter_df.shape

(201445, 19)

In [16]:
# excluding the dates before 8/1/16 reduced the number of observations by 163,273 or ~45%

filter_df.shape[0] - all_df.shape[0], ((filter_df.shape[0] - all_df.shape[0]) / all_df.shape[0]) * 100

(-163273, -44.76691580892635)

In [17]:
# odd that they had ads after election day (runoff elections e.g., Louisiana) -- will filter out post 11/8/16

filter_df['start_time'].min(), filter_df['start_time'].max()

(Timestamp('2016-08-01 00:35:22'), Timestamp('2016-11-29 18:44:37'))

In [18]:
filter_df = filter_df[filter_df['start_time'] <= datetime(2016, 11, 8, 23, 59, 59)]
filter_df.head(2)

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created
0,1,5643,KLAS,"Las Vegas, NV",8 News Now at 5 PM,news,2016-09-09 00:12:59,2016-09-09 00:13:29,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32
1,2,5643,WNCN,"Raleigh-Durham-Fayetteville, NC",North Carolina News at 500PM,news,2016-09-06 21:58:25,2016-09-06 21:58:55,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32


In [19]:
# excluding ads after 11/8/16 - removed another 162

filter_df.shape

(201283, 19)

In [20]:
# now use boolean selection to limit observations to presidential race only

filter_df = filter_df[filter_df['race'] == 'PRES']
filter_df.head(2)

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created
0,1,5643,KLAS,"Las Vegas, NV",8 News Now at 5 PM,news,2016-09-09 00:12:59,2016-09-09 00:13:29,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32
1,2,5643,WNCN,"Raleigh-Durham-Fayetteville, NC",North Carolina News at 500PM,news,2016-09-06 21:58:25,2016-09-06 21:58:55,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32


In [21]:
# confirming that mask worked

filter_df['race'].unique()

array(['PRES'], dtype=object)

In [22]:
filter_df.shape

(85152, 19)

In [23]:
# removed another 116,131 observations by limiting to the presidential race

201283 - 85152

116131

### Coding Plan II - Initial Feature Engineering

- create `raw_pres_df` and pickle from `filter_df.copy()`
- new features: `ad_length`, `time_from_creation` (in days)
- split `location` to create separate `metro_area` and `state` features
- `day_of_week` feature created from `start_time` columns
- `subject_count` feature -- number of subjects mentioned per ad -- derived from `subjects` column
- note potential gotcha on timestamps -- do times reflect local time of recording, UTC or time zone of computer

In [24]:
raw_pres_df = filter_df.copy()

In [25]:
# raw_pres_df.to_pickle('data/pres_raw_data.pickle')

In [26]:
pres_df = raw_pres_df.copy()

In [27]:
pres_df.head(2)

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created
0,1,5643,KLAS,"Las Vegas, NV",8 News Now at 5 PM,news,2016-09-09 00:12:59,2016-09-09 00:13:29,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32
1,2,5643,WNCN,"Raleigh-Durham-Fayetteville, NC",North Carolina News at 500PM,news,2016-09-06 21:58:25,2016-09-06 21:58:55,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32


### Creating `ad_length` feature

In [28]:
pres_df['ad_length_tmp'] = pres_df['end_time'] - pres_df['start_time']
pres_df['ad_length_tmp'].head(10)

0   00:00:30
1   00:00:30
2   00:00:30
3   00:00:30
4   00:00:30
5   00:00:30
6   00:00:30
7   00:00:30
8   00:00:30
9   00:00:30
Name: ad_length_tmp, dtype: timedelta64[ns]

In [29]:
# converting to seconds for readability

pres_df['ad_length_tmp'] = pres_df['ad_length_tmp'] / np.timedelta64(1, 's')
pres_df['ad_length_tmp'].head(10)

0   30.00
1   30.00
2   30.00
3   30.00
4   30.00
5   30.00
6   30.00
7   30.00
8   30.00
9   30.00
Name: ad_length_tmp, dtype: float64

In [30]:
# satisfied with conversion - make tmp column permanent - just practicing drop -- could have just renamed column

pres_df['ad_length'] = pres_df['ad_length_tmp']

In [31]:
pres_df.drop('ad_length_tmp', inplace=True, axis=1)
pres_df.head()

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created,ad_length
0,1,5643,KLAS,"Las Vegas, NV",8 News Now at 5 PM,news,2016-09-09 00:12:59,2016-09-09 00:13:29,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00
1,2,5643,WNCN,"Raleigh-Durham-Fayetteville, NC",North Carolina News at 500PM,news,2016-09-06 21:58:25,2016-09-06 21:58:55,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00
2,3,5643,KLAS,"Las Vegas, NV",Face the Nation,news,2016-09-04 16:24:25,2016-09-04 16:24:55,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00
3,4,5643,WFLA,"Tampa-St. Petersburg, FL",Days of Our Lives,not news,2016-08-30 17:59:20,2016-08-30 17:59:50,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00
4,5,5643,KSNV,"Las Vegas, NV",The Tonight Show Starring Jimmy Fallon,news,2016-09-06 07:02:22,2016-09-06 07:02:52,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00


In [32]:
pres_df['ad_length'].describe()

count   85,152.00
mean        33.19
std         11.20
min         15.00
25%         30.00
50%         30.00
75%         30.00
max        184.00
Name: ad_length, dtype: float64

### Creating `time_from_creation` feature (days)

In [33]:
pres_df['time_from_creation_tmp'] = pres_df['start_time'] - pres_df['date_created']
pres_df['time_from_creation_tmp'].head(10)

0    -4 days +09:23:27
1    -6 days +07:08:53
2    -8 days +01:34:53
3   -13 days +03:09:48
4    -7 days +16:12:50
5   -13 days +05:05:42
6   -14 days +22:32:58
7    -6 days +09:55:28
8   -13 days +07:23:27
9   -14 days +15:03:29
Name: time_from_creation_tmp, dtype: timedelta64[ns]

In [34]:
pres_df['time_from_creation_tmp'].tail(10)

364039   -21 days +17:33:53
364040   -27 days +23:02:53
364041   -20 days +00:04:06
364042   -27 days +18:48:57
364043   -27 days +07:47:33
364044   -25 days +17:18:46
364045   -21 days +18:32:59
364046   -20 days +06:41:44
364183   -21 days +05:24:29
364184   -21 days +12:40:50
Name: time_from_creation_tmp, dtype: timedelta64[ns]

In [35]:
# this feature does not seem to make sense, perhaps 'date_created' is the date the ad record was created -
# not when it was literally created -- therefore going to to drop this created column

pres_df.drop('time_from_creation_tmp', inplace=True, axis=1)
pres_df.head()

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created,ad_length
0,1,5643,KLAS,"Las Vegas, NV",8 News Now at 5 PM,news,2016-09-09 00:12:59,2016-09-09 00:13:29,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00
1,2,5643,WNCN,"Raleigh-Durham-Fayetteville, NC",North Carolina News at 500PM,news,2016-09-06 21:58:25,2016-09-06 21:58:55,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00
2,3,5643,KLAS,"Las Vegas, NV",Face the Nation,news,2016-09-04 16:24:25,2016-09-04 16:24:55,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00
3,4,5643,WFLA,"Tampa-St. Petersburg, FL",Days of Our Lives,not news,2016-08-30 17:59:20,2016-08-30 17:59:50,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00
4,5,5643,KSNV,"Las Vegas, NV",The Tonight Show Starring Jimmy Fallon,news,2016-09-06 07:02:22,2016-09-06 07:02:52,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00


### Creating `day_of_week` feature - day of the week the ad aired

In [36]:
pres_df['day_of_week'] = pres_df['start_time'].map(lambda x: x.strftime("%A"))
pres_df.head()

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created,ad_length,day_of_week
0,1,5643,KLAS,"Las Vegas, NV",8 News Now at 5 PM,news,2016-09-09 00:12:59,2016-09-09 00:13:29,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Friday
1,2,5643,WNCN,"Raleigh-Durham-Fayetteville, NC",North Carolina News at 500PM,news,2016-09-06 21:58:25,2016-09-06 21:58:55,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Tuesday
2,3,5643,KLAS,"Las Vegas, NV",Face the Nation,news,2016-09-04 16:24:25,2016-09-04 16:24:55,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Sunday
3,4,5643,WFLA,"Tampa-St. Petersburg, FL",Days of Our Lives,not news,2016-08-30 17:59:20,2016-08-30 17:59:50,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Tuesday
4,5,5643,KSNV,"Las Vegas, NV",The Tonight Show Starring Jimmy Fallon,news,2016-09-06 07:02:22,2016-09-06 07:02:52,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Tuesday


### Creating `metro_area` and `state` features

- need to split strings in `location` and then access separately via list slicing

In [37]:
# testing proof of concept -- going to split 'location' and then access separately

pres_df['location'][0].split(',')

['Las Vegas', ' NV']

In [38]:
# checking for NaNs in `location`

# going to have an issue because there are NaN's -- cannot split NaN's -- going to get an error

pres_df['location'].unique()

array(['Las Vegas, NV', 'Raleigh-Durham-Fayetteville,  NC',
       'Tampa-St. Petersburg, FL', 'Cleveland, Ohio',
       'Ceder Rapids-Waterloo-Iowa City-Dublin, Iowa', 'Philadelphia, PA',
       'Denver, CO', 'Milwaukee, WI', 'San Francisco-Oakland-San Jose, CA',
       'Phoenix-Prescott, AZ', 'Boston, MA/Manchester, NH',
       'Washington, DC/Hagerstown, MD', 'New York City, NY', nan], dtype=object)

In [39]:
# How many NaN's -- pretty good case to drop observations with location NaN's as only 25 out of 85K observations

pres_df['location'].isnull().sum(), pres_df.shape

(25, (85152, 21))

In [40]:
# dropping rows where 'location' NaNs

pres_df.dropna(subset=['location'], inplace=True)
pres_df.shape

(85127, 21)

In [41]:
# confirming removal of NaNs in 'location'

pres_df['location'].unique()

array(['Las Vegas, NV', 'Raleigh-Durham-Fayetteville,  NC',
       'Tampa-St. Petersburg, FL', 'Cleveland, Ohio',
       'Ceder Rapids-Waterloo-Iowa City-Dublin, Iowa', 'Philadelphia, PA',
       'Denver, CO', 'Milwaukee, WI', 'San Francisco-Oakland-San Jose, CA',
       'Phoenix-Prescott, AZ', 'Boston, MA/Manchester, NH',
       'Washington, DC/Hagerstown, MD', 'New York City, NY'], dtype=object)

### Now creating `split_location_tmp`

In [42]:
pres_df['split_location_tmp'] = pres_df['location'].map(lambda x: x.split(','))
pres_df.head(2)

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created,ad_length,day_of_week,split_location_tmp
0,1,5643,KLAS,"Las Vegas, NV",8 News Now at 5 PM,news,2016-09-09 00:12:59,2016-09-09 00:13:29,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Friday,"[Las Vegas, NV]"
1,2,5643,WNCN,"Raleigh-Durham-Fayetteville, NC",North Carolina News at 500PM,news,2016-09-06 21:58:25,2016-09-06 21:58:55,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Tuesday,"[Raleigh-Durham-Fayetteville, NC]"


In [43]:
pres_df['split_location_tmp'].head()

0                       [Las Vegas,  NV]
1    [Raleigh-Durham-Fayetteville,   NC]
2                       [Las Vegas,  NV]
3            [Tampa-St. Petersburg,  FL]
4                       [Las Vegas,  NV]
Name: split_location_tmp, dtype: object

### Use `split_location_tmp` to create `metro_area` 

In [44]:
pres_df['metro_area'] = pres_df['split_location_tmp'].map(lambda x: x[0])
pres_df['metro_area'].head()

0                      Las Vegas
1    Raleigh-Durham-Fayetteville
2                      Las Vegas
3           Tampa-St. Petersburg
4                      Las Vegas
Name: metro_area, dtype: object

In [45]:
pres_df['metro_area'].tail()

364042    Cleveland
364043    Cleveland
364044    Cleveland
364045    Cleveland
364046    Cleveland
Name: metro_area, dtype: object

In [46]:
pres_df['metro_area'].unique()

array(['Las Vegas', 'Raleigh-Durham-Fayetteville', 'Tampa-St. Petersburg',
       'Cleveland', 'Ceder Rapids-Waterloo-Iowa City-Dublin',
       'Philadelphia', 'Denver', 'Milwaukee',
       'San Francisco-Oakland-San Jose', 'Phoenix-Prescott', 'Boston',
       'Washington', 'New York City'], dtype=object)

In [47]:
pres_df['metro_area'].value_counts()

Tampa-St. Petersburg                      13248
Philadelphia                              11949
Las Vegas                                 10953
Cleveland                                 10604
San Francisco-Oakland-San Jose             7546
Raleigh-Durham-Fayetteville                6731
Ceder Rapids-Waterloo-Iowa City-Dublin     5295
Boston                                     5008
Denver                                     4710
Milwaukee                                  4097
Phoenix-Prescott                           3244
Washington                                 1695
New York City                                47
Name: metro_area, dtype: int64

### Using the same methodology, creating `state` feature

In [48]:
pres_df['state'] = pres_df['split_location_tmp'].map(lambda x: x[1])
pres_df['state'].head()

0      NV
1      NC
2      NV
3      FL
4      NV
Name: state, dtype: object

In [49]:
pres_df['state'].unique()

array([' NV', '  NC', ' FL', ' Ohio', ' Iowa', ' PA', ' CO', ' WI', ' CA',
       ' AZ', ' MA/Manchester', ' DC/Hagerstown', ' NY'], dtype=object)

### Notes:

- a little cleanup may be required here using some domain knowledge

- Boston media market most likely used to target New Hampshire voters -- convert "MA/Manchester" to New Hampshire

- DC/Hagerstown media buys used to influence Virgina state race

- could ignore and go with current labels (as we know what they are), but going to change

- while at it, could change CA to "National" as they were network buys (San Francisco Market)

- a little uncertainty over why they were NY observations -- could ads shown in NYC stations be targeted at eastern PA??

- there is a case for dropping the NY observations as they are only 47 out of 85K - did end up dropping NY observations in another notebook -- used non-NY dataset to generate visualizations

In [50]:
# using `.str.replace(A, B)` to get all states in same 2 letter abbreviation format

pres_df['state'] = pres_df['state'].str.replace(' MA/Manchester', 'NH')
pres_df['state'] = pres_df['state'].str.replace(' DC/Hagerstown', 'VA')
pres_df['state'] = pres_df['state'].str.replace(' Ohio', 'OH')
pres_df['state'] = pres_df['state'].str.replace(' Iowa', 'IA')
pres_df['state'] = pres_df['state'].str.replace(' CA', 'National')

In [51]:
# getting rid of unnecessary whitespace in the labels - could be a gotcha later on

pres_df['state'] = pres_df['state'].map(lambda x: x.strip())

In [52]:
pres_df['state'].value_counts()

FL          13248
PA          11949
NV          10953
OH          10604
National     7546
NC           6731
IA           5295
NH           5008
CO           4710
WI           4097
AZ           3244
VA           1695
NY             47
Name: state, dtype: int64

In [53]:
# doing some housekeepin on the DataFrame by dropping the 'split_location_tmp' column

pres_df.drop('split_location_tmp', inplace=True, axis=1)
pres_df.head(2)

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created,ad_length,day_of_week,metro_area,state
0,1,5643,KLAS,"Las Vegas, NV",8 News Now at 5 PM,news,2016-09-09 00:12:59,2016-09-09 00:13:29,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Friday,Las Vegas,NV
1,2,5643,WNCN,"Raleigh-Durham-Fayetteville, NC",North Carolina News at 500PM,news,2016-09-06 21:58:25,2016-09-06 21:58:55,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Tuesday,Raleigh-Durham-Fayetteville,NC


In [54]:
pres_df.shape

(85127, 23)

### Creating `subject_count` feature 

- use same basic methodology as in creation of `metro_area` and `state` features from splitting `location`

In [55]:
# proof of concept

len(pres_df['subjects'][0].split())

3

In [56]:
# checking for NaNs

pres_df['subjects'].isnull().sum()

591

In [57]:
pres_df.ix[362865] # how to get row info by index number

id                                                          462751
wp_identifier                                                 8248
network                                                       WUVP
location                                          Philadelphia, PA
program                                          Despierta America
program_type                                                  news
start_time                                     2016-10-19 12:33:03
end_time                                       2016-10-19 12:34:18
archive_id                              PolAd_HillaryClinton_ia316
embed_url        https://archive.org/embed/PolAd_HillaryClinton...
sponsors                                        America Rising PAC
sponsor_types                                            Super PAC
race                                                          PRES
cycle                                                     2,016.00
subjects                                                      

In [58]:
pres_df.ix[362865]['subjects'] # how to get row info by index number and column

nan

In [59]:
pres_df['subjects'].nunique() # unique combinations of ad topics

212

In [60]:
pres_df['subjects'].describe()

count                                    84536
unique                                     212
top       Women, Candidate Biography, Children
freq                                      4184
Name: subjects, dtype: object

In [61]:
# writing a function to split the cell, but keep NaNs - use apply + function to transform column

def subject_count(cell):
    try:
        s_count = len((cell).split(','))
    except AttributeError:
        return np.nan
    return s_count

# could have used this pattern from asynch

#if cell is np.nan:
        #return np.nan -- if it's NaN, leave it
    
# there is a gotcha that np.nan == np.nan returns FALSE, but if you use 'np.nan is np.nan' get TRUE, 
# or use try and except as we di here

In [62]:
# applying subject_count function to `subject_count_tmp`

pres_df['subject_count_tmp'] = pres_df['subjects'].apply(subject_count) # can use apply instead of map
pres_df.head()

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created,ad_length,day_of_week,metro_area,state,subject_count_tmp
0,1,5643,KLAS,"Las Vegas, NV",8 News Now at 5 PM,news,2016-09-09 00:12:59,2016-09-09 00:13:29,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Friday,Las Vegas,NV,3.00
1,2,5643,WNCN,"Raleigh-Durham-Fayetteville, NC",North Carolina News at 500PM,news,2016-09-06 21:58:25,2016-09-06 21:58:55,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Tuesday,Raleigh-Durham-Fayetteville,NC,3.00
2,3,5643,KLAS,"Las Vegas, NV",Face the Nation,news,2016-09-04 16:24:25,2016-09-04 16:24:55,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Sunday,Las Vegas,NV,3.00
3,4,5643,WFLA,"Tampa-St. Petersburg, FL",Days of Our Lives,not news,2016-08-30 17:59:20,2016-08-30 17:59:50,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Tuesday,Tampa-St. Petersburg,FL,3.00
4,5,5643,KSNV,"Las Vegas, NV",The Tonight Show Starring Jimmy Fallon,news,2016-09-06 07:02:22,2016-09-06 07:02:52,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Tuesday,Las Vegas,NV,3.00


In [63]:
pres_df['subject_count_tmp'].value_counts(dropna=False)

# this is a gotcha - have to put dropna=False in order to get the NaN's in value_counts
# want to keep the NaN's b/c can do math on the series (if convert NaNs to strings cannot do so)

3.00    20787
4.00    19869
2.00    15428
5.00    11875
6.00     5942
1.00     4716
7.00     3827
8.00     2092
nan       591
Name: subject_count_tmp, dtype: int64

In [64]:
type(pres_df['subject_count_tmp'][0])

numpy.float64

In [65]:
pres_df['subject_count_tmp'].mean(), pres_df['subject_count_tmp'].median()

(3.7375910854547176, 4.0)

In [66]:
pres_df.shape

(85127, 24)

In [67]:
# since happy with subject_count - rename the column - get rid of the tmp tag

pres_df = pres_df.rename(columns={'subject_count_tmp': 'subject_count'})
pres_df.head(2)

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created,ad_length,day_of_week,metro_area,state,subject_count
0,1,5643,KLAS,"Las Vegas, NV",8 News Now at 5 PM,news,2016-09-09 00:12:59,2016-09-09 00:13:29,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Friday,Las Vegas,NV,3.00
1,2,5643,WNCN,"Raleigh-Durham-Fayetteville, NC",North Carolina News at 500PM,news,2016-09-06 21:58:25,2016-09-06 21:58:55,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Tuesday,Raleigh-Durham-Fayetteville,NC,3.00


In [68]:
# can also rename columns inplace

pres_df.rename(columns={'subject_count': 'subject_count_test'}, inplace=True)
pres_df.head(2)

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created,ad_length,day_of_week,metro_area,state,subject_count_test
0,1,5643,KLAS,"Las Vegas, NV",8 News Now at 5 PM,news,2016-09-09 00:12:59,2016-09-09 00:13:29,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Friday,Las Vegas,NV,3.00
1,2,5643,WNCN,"Raleigh-Durham-Fayetteville, NC",North Carolina News at 500PM,news,2016-09-06 21:58:25,2016-09-06 21:58:55,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Tuesday,Raleigh-Durham-Fayetteville,NC,3.00


In [69]:
# changing back to `subject_count`

pres_df.rename(columns={'subject_count_test': 'subject_count'}, inplace=True)
pres_df.head(2)

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created,ad_length,day_of_week,metro_area,state,subject_count
0,1,5643,KLAS,"Las Vegas, NV",8 News Now at 5 PM,news,2016-09-09 00:12:59,2016-09-09 00:13:29,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Friday,Las Vegas,NV,3.00
1,2,5643,WNCN,"Raleigh-Durham-Fayetteville, NC",North Carolina News at 500PM,news,2016-09-06 21:58:25,2016-09-06 21:58:55,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Tuesday,Raleigh-Durham-Fayetteville,NC,3.00


### Pickling `pres_df` to preserve this intemediate stage of the EDA/Cleaning/Feature Engineering Process

- old notebook `Political_Ads_EDA_v3` ended here by pickling `pres_df` at this stage of the process
- `Political_Ads_Visualizations_v3` continued the EDA/Cleaning/Feature Engineering process by reading in the pickled data filename below

In [70]:
# comment over to prevent overwrite

# pres_df.to_pickle('data/pres_sorted_data.pickle')

### Creating `hour_aired` - hour of the day (military time) that ad aired

In [71]:
pres_df['hour_aired'] = pres_df['start_time'].map(lambda x: int(x.strftime("%H")))
pres_df.head(2)

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created,ad_length,day_of_week,metro_area,state,subject_count,hour_aired
0,1,5643,KLAS,"Las Vegas, NV",8 News Now at 5 PM,news,2016-09-09 00:12:59,2016-09-09 00:13:29,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Friday,Las Vegas,NV,3.00,0
1,2,5643,WNCN,"Raleigh-Durham-Fayetteville, NC",North Carolina News at 500PM,news,2016-09-06 21:58:25,2016-09-06 21:58:55,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Tuesday,Raleigh-Durham-Fayetteville,NC,3.00,21


In [72]:
pres_df['hour_aired'].dtypes

dtype('int64')

In [73]:
pres_df['hour_aired'].value_counts()

23    6836
22    6311
12    5163
21    5098
13    4848
3     4663
11    4095
16    3976
10    3848
4     3753
14    3702
2     3483
20    3241
15    3213
0     3134
9     2897
19    2738
1     2702
5     2631
17    2405
18    2191
6     1709
8     1540
7      950
Name: hour_aired, dtype: int64

In [74]:
len(pres_df['hour_aired'].value_counts()) # sanity check - should be 24

24

In [75]:
pres_df.head()

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created,ad_length,day_of_week,metro_area,state,subject_count,hour_aired
0,1,5643,KLAS,"Las Vegas, NV",8 News Now at 5 PM,news,2016-09-09 00:12:59,2016-09-09 00:13:29,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Friday,Las Vegas,NV,3.00,0
1,2,5643,WNCN,"Raleigh-Durham-Fayetteville, NC",North Carolina News at 500PM,news,2016-09-06 21:58:25,2016-09-06 21:58:55,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Tuesday,Raleigh-Durham-Fayetteville,NC,3.00,21
2,3,5643,KLAS,"Las Vegas, NV",Face the Nation,news,2016-09-04 16:24:25,2016-09-04 16:24:55,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Sunday,Las Vegas,NV,3.00,16
3,4,5643,WFLA,"Tampa-St. Petersburg, FL",Days of Our Lives,not news,2016-08-30 17:59:20,2016-08-30 17:59:50,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Tuesday,Tampa-St. Petersburg,FL,3.00,17
4,5,5643,KSNV,"Las Vegas, NV",The Tonight Show Starring Jimmy Fallon,news,2016-09-06 07:02:22,2016-09-06 07:02:52,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Tuesday,Las Vegas,NV,3.00,7


In [76]:
pres_df.shape

(85127, 25)

## Setting DataFrame Index to `start_date`

In [77]:
# Making a copy before setting index to `start_date`

pres_date_df = pres_df.copy()
pres_date_df.head(2)

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created,ad_length,day_of_week,metro_area,state,subject_count,hour_aired
0,1,5643,KLAS,"Las Vegas, NV",8 News Now at 5 PM,news,2016-09-09 00:12:59,2016-09-09 00:13:29,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Friday,Las Vegas,NV,3.00,0
1,2,5643,WNCN,"Raleigh-Durham-Fayetteville, NC",North Carolina News at 500PM,news,2016-09-06 21:58:25,2016-09-06 21:58:55,PolAd_HillaryClinton_f1h3j,https://archive.org/embed/PolAd_HillaryClinton...,Hillary for America,Multiple,PRES,"2,016.00","Energy, China, Jobs",Hillary Clinton,campaign,pro,2016-09-12 14:49:32,30.00,Tuesday,Raleigh-Durham-Fayetteville,NC,3.00,21


In [78]:
# start_time column was not sorted -- sorting here in ascending order

pres_date_df.sort_values('start_time', ascending=True, inplace=True)
pres_date_df.head(10)

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created,ad_length,day_of_week,metro_area,state,subject_count,hour_aired
207673,261456,6035,FOXNEWSW,"San Francisco-Oakland-San Jose, CA",On the Record With Greta Van Susteren,news,2016-08-01 01:19:59,2016-08-01 01:21:04,PolAd_HillaryClinton_4drmd,https://archive.org/embed/PolAd_HillaryClinton...,Reform America Fund,Super PAC,PRES,"2,016.00",Candidate Biography,Hillary Clinton,campaign,con,2016-09-21 10:40:12,65.00,Monday,San Francisco-Oakland-San Jose,National,1.00,1
207667,261450,6035,FOXNEWSW,"San Francisco-Oakland-San Jose, CA",Fox News Sunday With Chris Wallace,news,2016-08-01 06:20:04,2016-08-01 06:21:09,PolAd_HillaryClinton_4drmd,https://archive.org/embed/PolAd_HillaryClinton...,Reform America Fund,Super PAC,PRES,"2,016.00",Candidate Biography,Hillary Clinton,campaign,con,2016-09-21 10:40:12,65.00,Monday,San Francisco-Oakland-San Jose,National,1.00,6
34380,48222,4176,WCAU,"Philadelphia, PA",NBC10 News Today at 430am,news,2016-08-01 08:39:49,2016-08-01 08:40:21,PolAd_DonaldTrump_99k4w,https://archive.org/embed/PolAd_DonaldTrump_99k4w,Hillary for America,Multiple,PRES,"2,016.00","Candidate Biography, Foreign Policy, Nuclear, ...",Donald Trump,campaign,mixed,2016-09-12 14:51:51,32.00,Monday,Philadelphia,PA,4.00,8
34553,48395,4176,KYW,"Philadelphia, PA",Eyewitness News at 430,news,2016-08-01 08:48:14,2016-08-01 08:48:46,PolAd_DonaldTrump_99k4w,https://archive.org/embed/PolAd_DonaldTrump_99k4w,Hillary for America,Multiple,PRES,"2,016.00","Candidate Biography, Foreign Policy, Nuclear, ...",Donald Trump,campaign,mixed,2016-09-12 14:51:51,32.00,Monday,Philadelphia,PA,4.00,8
279413,378441,7459,MSNBCW,"San Francisco-Oakland-San Jose, CA",First Look,news,2016-08-01 09:09:45,2016-08-01 09:10:15,PolAd_HillaryClinton_4gvic,https://archive.org/embed/PolAd_HillaryClinton...,Make America Number One,Super PAC,PRES,"2,016.00",Candidate Biography,Hillary Clinton,campaign,con,2016-10-21 10:40:01,30.00,Monday,San Francisco-Oakland-San Jose,National,1.00,9
34552,48394,4176,WCAU,"Philadelphia, PA",NBC10 News Today at 5am,news,2016-08-01 09:24:05,2016-08-01 09:24:37,PolAd_DonaldTrump_99k4w,https://archive.org/embed/PolAd_DonaldTrump_99k4w,Hillary for America,Multiple,PRES,"2,016.00","Candidate Biography, Foreign Policy, Nuclear, ...",Donald Trump,campaign,mixed,2016-09-12 14:51:51,32.00,Monday,Philadelphia,PA,4.00,9
279420,378448,7459,MSNBCW,"San Francisco-Oakland-San Jose, CA",Way Too Early,news,2016-08-01 09:39:09,2016-08-01 09:39:39,PolAd_HillaryClinton_4gvic,https://archive.org/embed/PolAd_HillaryClinton...,Make America Number One,Super PAC,PRES,"2,016.00",Candidate Biography,Hillary Clinton,campaign,con,2016-10-21 10:40:01,30.00,Monday,San Francisco-Oakland-San Jose,National,1.00,9
207674,261457,6035,MSNBCW,"San Francisco-Oakland-San Jose, CA",Morning Joe,news,2016-08-01 10:26:51,2016-08-01 10:27:56,PolAd_HillaryClinton_4drmd,https://archive.org/embed/PolAd_HillaryClinton...,Reform America Fund,Super PAC,PRES,"2,016.00",Candidate Biography,Hillary Clinton,campaign,con,2016-09-21 10:40:12,65.00,Monday,San Francisco-Oakland-San Jose,National,1.00,10
279418,378446,7459,MSNBCW,"San Francisco-Oakland-San Jose, CA",Morning Joe,news,2016-08-01 10:27:50,2016-08-01 10:28:20,PolAd_HillaryClinton_4gvic,https://archive.org/embed/PolAd_HillaryClinton...,Make America Number One,Super PAC,PRES,"2,016.00",Candidate Biography,Hillary Clinton,campaign,con,2016-10-21 10:40:01,30.00,Monday,San Francisco-Oakland-San Jose,National,1.00,10
34551,48393,4176,WTXF,"Philadelphia, PA",Good Day Philadelphia at 6A,news,2016-08-01 10:29:58,2016-08-01 10:30:30,PolAd_DonaldTrump_99k4w,https://archive.org/embed/PolAd_DonaldTrump_99k4w,Hillary for America,Multiple,PRES,"2,016.00","Candidate Biography, Foreign Policy, Nuclear, ...",Donald Trump,campaign,mixed,2016-09-12 14:51:51,32.00,Monday,Philadelphia,PA,4.

### Now setting `start_time` as the index for `pres_date_df`

In [79]:
pres_date_df.set_index('start_time', inplace=True, drop=False) # could have dropped, but in this case kept it
pres_date_df.head(20)

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created,ad_length,day_of_week,metro_area,state,subject_count,hour_aired
start_time,,,,,,,,,,,,,,,,,,,,,,,,,
2016-08-01 01:19:59,261456,6035,FOXNEWSW,"San Francisco-Oakland-San Jose, CA",On the Record With Greta Van Susteren,news,2016-08-01 01:19:59,2016-08-01 01:21:04,PolAd_HillaryClinton_4drmd,https://archive.org/embed/PolAd_HillaryClinton...,Reform America Fund,Super PAC,PRES,"2,016.00",Candidate Biography,Hillary Clinton,campaign,con,2016-09-21 10:40:12,65.00,Monday,San Francisco-Oakland-San Jose,National,1.00,1
2016-08-01 06:20:04,261450,6035,FOXNEWSW,"San Francisco-Oakland-San Jose, CA",Fox News Sunday With Chris Wallace,news,2016-08-01 06:20:04,2016-08-01 06:21:09,PolAd_HillaryClinton_4drmd,https://archive.org/embed/PolAd_HillaryClinton...,Reform America Fund,Super PAC,PRES,"2,016.00",Candidate Biography,Hillary Clinton,campaign,con,2016-09-21 10:40:12,65.00,Monday,San Francisco-Oakland-San Jose,National,1.00,6
2016-08-01 08:39:49,48222,4176,WCAU,"Philadelphia, PA",NBC10 News Today at 430am,news,2016-08-01 08:39:49,2016-08-01 08:40:21,PolAd_DonaldTrump_99k4w,https://archive.org/embed/PolAd_DonaldTrump_99k4w,Hillary for America,Multiple,PRES,"2,016.00","Candidate Biography, Foreign Policy, Nuclear, ...",Donald Trump,campaign,mixed,2016-09-12 14:51:51,32.00,Monday,Philadelphia,PA,4.00,8
2016-08-01 08:48:14,48395,4176,KYW,"Philadelphia, PA",Eyewitness News at 430,news,2016-08-01 08:48:14,2016-08-01 08:48:46,PolAd_DonaldTrump_99k4w,https://archive.org/embed/PolAd_DonaldTrump_99k4w,Hillary for America,Multiple,PRES,"2,016.00","Candidate Biography, Foreign Policy, Nuclear, ...",Donald Trump,campaign,mixed,2016-09-12 14:51:51,32.00,Monday,Philadelphia,PA,4.00,8
2016-08-01 09:09:45,378441,7459,MSNBCW,"San Francisco-Oakland-San Jose, CA",First Look,news,2016-08-01 09:09:45,2016-08-01 09:10:15,PolAd_HillaryClinton_4gvic,https://archive.org/embed/PolAd_HillaryClinton...,Make America Number One,Super PAC,PRES,"2,016.00",Candidate Biography,Hillary Clinton,campaign,con,2016-10-21 10:40:01,30.00,Monday,San Francisco-Oakland-San Jose,National,1.00,9
2016-08-01 09:24:05,48394,4176,WCAU,"Philadelphia, PA",NBC10 News Today at 5am,news,2016-08-01 09:24:05,2016-08-01 09:24:37,PolAd_DonaldTrump_99k4w,https://archive.org/embed/PolAd_DonaldTrump_99k4w,Hillary for America,Multiple,PRES,"2,016.00","Candidate Biography, Foreign Policy, Nuclear, ...",Donald Trump,campaign,mixed,2016-09-12 14:51:51,32.00,Monday,Philadelphia,PA,4.00,9
2016-08-01 09:39:09,378448,7459,MSNBCW,"San Francisco-Oakland-San Jose, CA",Way Too Early,news,2016-08-01 09:39:09,2016-08-01 09:39:39,PolAd_HillaryClinton_4gvic,https://archive.org/embed/PolAd_HillaryClinton...,Make America Number One,Super PAC,PRES,"2,016.00",Candidate Biography,Hillary Clinton,campaign,con,2016-10-21 10:40:01,30.00,Monday,San Francisco-Oakland-San Jose,National,1.00,9
2016-08-01 10:26:51,261457,6035,MSNBCW,"San Francisco-Oakland-San Jose, CA",Morning Joe,news,2016-08-01 10:26:51,2016-08-01 10:27:56,PolAd_HillaryClinton_4drmd,https://archive.org/embed/PolAd_HillaryClinton...,Reform America Fund,Super PAC,PRES,"2,016.00",Candidate Biography,Hillary Clinton,campaign,con,2016-09-21 10:40:12,65.00,Monday,San Francisco-Oakland-San Jose,National,1.00,10
2016-08-01 10:27:50,378446,7459,MSNBCW,"San Francisco-Oakland-San Jose, CA",Morning Joe,news,2016-08-01 10:27:50,2016-08-01 10:28:20,PolAd_HillaryClinton_4gvic,https://archive.org/embed/PolAd_HillaryClinton...,Make America Number One,Super PAC,PRES,"2,016.00",Candidate Biography,Hillary Clinton,campaign,con,2016-10-21 10:40:01,30.00,Monday,San Francisco-Oakland-San Jose,National,1.00,10


## Creating `sponsor_class` feature -- cleaning up and parsing through `sponsors` 

- required in order to determine which candidate was responsible for airing the ad and can match candidate to ad message (con, pro, mixed, unknown)
- had to manually research the 45 unique sponsors in this dataset to determine which candidate they backed
- primary search tool was OpenSecrets.org and wikipedia.com
- even though `race` classified ad as for `PRES` - the manual inspection of the `sponsors` showed that there were ads sponsored by senatorial campaigns -- filtered out these observations and sponsors
- left in 3rd Party candidate observations -- these were easily filtered out in the time series analysis

In [80]:
pres_date_df['sponsors'].nunique()

45

In [81]:
sponsors_list = list(pres_date_df['sponsors'].unique())
sponsors_list

['Reform America Fund',
 'Hillary for America',
 'Make America Number One',
 'Rebuilding America Now',
 'NextGen California Action Committee',
 'Women Vote!',
 'Reclaim America PAC',
 'Priorities USA Action',
 'Donald J Trump For President',
 'Kirkpatrick For Senate',
 'Friends of Patrick Murphy',
 'MoveOn.org',
 'Our Principles PAC',
 'Correct The Record',
 'Marco Rubio For Senate 2016',
 'Republican National Cmte',
 'Gary Johnson 2016',
 'Great America PAC',
 'Stop Hillary PAC',
 'National Rifle Assn',
 'Ready for Hillary',
 'Democratic Senatorial Campaign Cmte',
 'Purple PAC',
 'Jill Stein for President',
 'Strickland for Senate',
 'Free the Delegates',
 'NARAL Pro-Choice America',
 'American Bridge 21st Century',
 'Hillary Victory Fund',
 'Democratic National Cmte',
 'Fenton Communications Inc',
 'America Rising PAC',
 'NextGen Climate Action',
 'Save the Day PAC',
 'Future45',
 'Donald J. Trump For President',
 'NRA Institute for Legislative Action',
 'iAmerica Action',
 'Internet

In [82]:
pres_date_df['sponsors'].isnull().sum() # gives you the number of NaN's since True = 1

0

### External research on `sponsors`

- Exported sponsors_list a csv file and used MS Excel to annotate with information from OpenSecrets.org and other sources 
- filename is `data/sponsors_list_annotated.csv`

In [83]:
# code for exporting `sponsors_list` as csv file - easier to work and annotate in excel
# commenting out to prevent overwrite of original version

# import csv

# with open('data/sponsors_list.csv', 'wt') as f:
    # writer = csv.writer(f)
    # for row in sponsors_list:
        # writer.writerow([row])

### Reading in Annotated Sponsors List File - `data/sponsors_list_final.csv`

- `Sponsor_Classification` manually determined via web searches

In [84]:
sponsors_df = pd.read_csv('data/sponsors_list_final.csv', 
                          header=None, 
                          names=['Sponsors', 'Sponsor_Classification'], 
                          index_col='Sponsors')
sponsors_df.head()

,Sponsor_Classification
Sponsors,
Reform America Fund,R_PAC
Hillary for America,Clinton
Make America Number One,R_PAC
Rebuilding America Now,R_PAC
NextGen California Action Committee,D_PAC


In [85]:
sponsors_df.values[0:5]

array([['R_PAC'],
       ['Clinton'],
       ['R_PAC'],
       ['R_PAC'],
       ['D_PAC']], dtype=object)

In [86]:
# How to access a cell via an index

sponsors_df['Sponsor_Classification']['Reform America Fund']

'R_PAC'

### Creating a mapping dictionary - use to map `Sponsors` to `Sponsor_Classification`

In [87]:
mapping_dict = {}

for row in sponsors_df.index:
    mapping_dict[row] = sponsors_df['Sponsor_Classification'][row]
    
mapping_dict

{'45 Committee': 'R_PAC',
 'America Rising PAC': 'R_PAC',
 'American Bridge 21st Century': 'D_PAC',
 'American Crossroads': 'R_PAC',
 'Correct The Record': 'D_PAC',
 'Democratic National Cmte': 'D_COM',
 'Democratic Senatorial Campaign Cmte': 'Other',
 'Donald J Trump For President': 'Trump',
 'Donald J. Trump For President': 'Trump',
 'Fenton Communications Inc': 'D_PAC',
 'Fifty Second Street Fund': 'D_PAC',
 'Free the Delegates': 'Other',
 'Friends of Patrick Murphy': 'Other',
 'Future45': 'R_PAC',
 'Gary Johnson 2016': 'Johnson',
 'Great America PAC': 'R_PAC',
 'Hillary Victory Fund': 'D_PAC',
 'Hillary for America': 'Clinton',
 'Internet Archive': 'Other',
 'Jill Stein for President': 'Stein',
 'Kirkpatrick For Senate': 'Other',
 'Local Voices': 'D_PAC',
 'Make America Number One': 'R_PAC',
 'Marco Rubio For Senate 2016': 'Other',
 'MoveOn.org': 'D_PAC',
 'NARAL Pro-Choice America': 'D_PAC',
 'NRA Institute for Legislative Action': 'R_PAC',
 'National Republican Senatorial Cmte': 

### Now use 'mapping_dict' to create a new column that you can sort and mask on 

- first make a copy of `pres_date_df` to be safe

In [88]:
cand_date_df = pres_date_df.copy()
cand_date_df.head()

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created,ad_length,day_of_week,metro_area,state,subject_count,hour_aired
start_time,,,,,,,,,,,,,,,,,,,,,,,,,
2016-08-01 01:19:59,261456,6035,FOXNEWSW,"San Francisco-Oakland-San Jose, CA",On the Record With Greta Van Susteren,news,2016-08-01 01:19:59,2016-08-01 01:21:04,PolAd_HillaryClinton_4drmd,https://archive.org/embed/PolAd_HillaryClinton...,Reform America Fund,Super PAC,PRES,"2,016.00",Candidate Biography,Hillary Clinton,campaign,con,2016-09-21 10:40:12,65.00,Monday,San Francisco-Oakland-San Jose,National,1.00,1
2016-08-01 06:20:04,261450,6035,FOXNEWSW,"San Francisco-Oakland-San Jose, CA",Fox News Sunday With Chris Wallace,news,2016-08-01 06:20:04,2016-08-01 06:21:09,PolAd_HillaryClinton_4drmd,https://archive.org/embed/PolAd_HillaryClinton...,Reform America Fund,Super PAC,PRES,"2,016.00",Candidate Biography,Hillary Clinton,campaign,con,2016-09-21 10:40:12,65.00,Monday,San Francisco-Oakland-San Jose,National,1.00,6
2016-08-01 08:39:49,48222,4176,WCAU,"Philadelphia, PA",NBC10 News Today at 430am,news,2016-08-01 08:39:49,2016-08-01 08:40:21,PolAd_DonaldTrump_99k4w,https://archive.org/embed/PolAd_DonaldTrump_99k4w,Hillary for America,Multiple,PRES,"2,016.00","Candidate Biography, Foreign Policy, Nuclear, ...",Donald Trump,campaign,mixed,2016-09-12 14:51:51,32.00,Monday,Philadelphia,PA,4.00,8
2016-08-01 08:48:14,48395,4176,KYW,"Philadelphia, PA",Eyewitness News at 430,news,2016-08-01 08:48:14,2016-08-01 08:48:46,PolAd_DonaldTrump_99k4w,https://archive.org/embed/PolAd_DonaldTrump_99k4w,Hillary for America,Multiple,PRES,"2,016.00","Candidate Biography, Foreign Policy, Nuclear, ...",Donald Trump,campaign,mixed,2016-09-12 14:51:51,32.00,Monday,Philadelphia,PA,4.00,8
2016-08-01 09:09:45,378441,7459,MSNBCW,"San Francisco-Oakland-San Jose, CA",First Look,news,2016-08-01 09:09:45,2016-08-01 09:10:15,PolAd_HillaryClinton_4gvic,https://archive.org/embed/PolAd_HillaryClinton...,Make America Number One,Super PAC,PRES,"2,016.00",Candidate Biography,Hillary Clinton,campaign,con,2016-10-21 10:40:01,30.00,Monday,San Francisco-Oakland-San Jose,National,1.00,9


### Using `map` and `mapping_dict` to create a new feature `sponsor_class`

In [89]:
cand_date_df['sponsor_class'] = cand_date_df['sponsors'].map(mapping_dict)
cand_date_df.head()

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created,ad_length,day_of_week,metro_area,state,subject_count,hour_aired,sponsor_class
start_time,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-08-01 01:19:59,261456,6035,FOXNEWSW,"San Francisco-Oakland-San Jose, CA",On the Record With Greta Van Susteren,news,2016-08-01 01:19:59,2016-08-01 01:21:04,PolAd_HillaryClinton_4drmd,https://archive.org/embed/PolAd_HillaryClinton...,Reform America Fund,Super PAC,PRES,"2,016.00",Candidate Biography,Hillary Clinton,campaign,con,2016-09-21 10:40:12,65.00,Monday,San Francisco-Oakland-San Jose,National,1.00,1,R_PAC
2016-08-01 06:20:04,261450,6035,FOXNEWSW,"San Francisco-Oakland-San Jose, CA",Fox News Sunday With Chris Wallace,news,2016-08-01 06:20:04,2016-08-01 06:21:09,PolAd_HillaryClinton_4drmd,https://archive.org/embed/PolAd_HillaryClinton...,Reform America Fund,Super PAC,PRES,"2,016.00",Candidate Biography,Hillary Clinton,campaign,con,2016-09-21 10:40:12,65.00,Monday,San Francisco-Oakland-San Jose,National,1.00,6,R_PAC
2016-08-01 08:39:49,48222,4176,WCAU,"Philadelphia, PA",NBC10 News Today at 430am,news,2016-08-01 08:39:49,2016-08-01 08:40:21,PolAd_DonaldTrump_99k4w,https://archive.org/embed/PolAd_DonaldTrump_99k4w,Hillary for America,Multiple,PRES,"2,016.00","Candidate Biography, Foreign Policy, Nuclear, ...",Donald Trump,campaign,mixed,2016-09-12 14:51:51,32.00,Monday,Philadelphia,PA,4.00,8,Clinton
2016-08-01 08:48:14,48395,4176,KYW,"Philadelphia, PA",Eyewitness News at 430,news,2016-08-01 08:48:14,2016-08-01 08:48:46,PolAd_DonaldTrump_99k4w,https://archive.org/embed/PolAd_DonaldTrump_99k4w,Hillary for America,Multiple,PRES,"2,016.00","Candidate Biography, Foreign Policy, Nuclear, ...",Donald Trump,campaign,mixed,2016-09-12 14:51:51,32.00,Monday,Philadelphia,PA,4.00,8,Clinton
2016-08-01 09:09:45,378441,7459,MSNBCW,"San Francisco-Oakland-San Jose, CA",First Look,news,2016-08-01 09:09:45,2016-08-01 09:10:15,PolAd_HillaryClinton_4gvic,https://archive.org/embed/PolAd_HillaryClinton...,Make America Number One,Super PAC,PRES,"2,016.00",Candidate Biography,Hillary Clinton,campaign,con,2016-10-21 10:40:01,30.00,Monday,San Francisco-Oakland-San Jose,National,1.00,9,R_PAC


In [90]:
cand_date_df.shape

(85127, 26)

In [91]:
cand_date_df['sponsor_class'].value_counts()

Clinton    37589
Trump      22918
D_PAC      14877
R_PAC       7602
Other       1778
D_COM        206
Johnson       82
R_COM         72
Stein          3
Name: sponsor_class, dtype: int64

## Creating the `party` feature

- first going to limit the dataset to only the relevant sponsor classes to presidential election 
- filter out `Other` in `sponsor_class` which refers to non-presidential campaign organizations (e.g., Marco Rubio's PAC)

In [92]:
cand_date_df = cand_date_df[cand_date_df['sponsor_class'] != 'Other']
cand_date_df.head(2)

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created,ad_length,day_of_week,metro_area,state,subject_count,hour_aired,sponsor_class
start_time,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-08-01 01:19:59,261456,6035,FOXNEWSW,"San Francisco-Oakland-San Jose, CA",On the Record With Greta Van Susteren,news,2016-08-01 01:19:59,2016-08-01 01:21:04,PolAd_HillaryClinton_4drmd,https://archive.org/embed/PolAd_HillaryClinton...,Reform America Fund,Super PAC,PRES,"2,016.00",Candidate Biography,Hillary Clinton,campaign,con,2016-09-21 10:40:12,65.00,Monday,San Francisco-Oakland-San Jose,National,1.00,1,R_PAC
2016-08-01 06:20:04,261450,6035,FOXNEWSW,"San Francisco-Oakland-San Jose, CA",Fox News Sunday With Chris Wallace,news,2016-08-01 06:20:04,2016-08-01 06:21:09,PolAd_HillaryClinton_4drmd,https://archive.org/embed/PolAd_HillaryClinton...,Reform America Fund,Super PAC,PRES,"2,016.00",Candidate Biography,Hillary Clinton,campaign,con,2016-09-21 10:40:12,65.00,Monday,San Francisco-Oakland-San Jose,National,1.00,6,R_PAC


In [93]:
cand_date_df.shape

(83349, 26)

In [94]:
# sanity check to ensure mask worked correctly - 1778

85127 - 83349

1778

### Using `map` and a `mapping dictionary` (`party_dict`) to create a new feature `party` from `sponsor_class`

In [95]:
cand_date_df['sponsor_class'].value_counts()

Clinton    37589
Trump      22918
D_PAC      14877
R_PAC       7602
D_COM        206
Johnson       82
R_COM         72
Stein          3
Name: sponsor_class, dtype: int64

In [96]:
party_dict = {'Clinton': 'D',
              'Trump': 'R',
              'D_PAC': 'D',
              'R_PAC': 'R',
              'D_COM': 'D',
              'Johnson': 'I',
              'R_COM': 'R',
              'Stein': 'I'
             }

In [97]:
# using map and 'party_dict'

cand_date_df['party'] = cand_date_df['sponsor_class'].map(party_dict)
cand_date_df.head()

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created,ad_length,day_of_week,metro_area,state,subject_count,hour_aired,sponsor_class,party
start_time,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-08-01 01:19:59,261456,6035,FOXNEWSW,"San Francisco-Oakland-San Jose, CA",On the Record With Greta Van Susteren,news,2016-08-01 01:19:59,2016-08-01 01:21:04,PolAd_HillaryClinton_4drmd,https://archive.org/embed/PolAd_HillaryClinton...,Reform America Fund,Super PAC,PRES,"2,016.00",Candidate Biography,Hillary Clinton,campaign,con,2016-09-21 10:40:12,65.00,Monday,San Francisco-Oakland-San Jose,National,1.00,1,R_PAC,R
2016-08-01 06:20:04,261450,6035,FOXNEWSW,"San Francisco-Oakland-San Jose, CA",Fox News Sunday With Chris Wallace,news,2016-08-01 06:20:04,2016-08-01 06:21:09,PolAd_HillaryClinton_4drmd,https://archive.org/embed/PolAd_HillaryClinton...,Reform America Fund,Super PAC,PRES,"2,016.00",Candidate Biography,Hillary Clinton,campaign,con,2016-09-21 10:40:12,65.00,Monday,San Francisco-Oakland-San Jose,National,1.00,6,R_PAC,R
2016-08-01 08:39:49,48222,4176,WCAU,"Philadelphia, PA",NBC10 News Today at 430am,news,2016-08-01 08:39:49,2016-08-01 08:40:21,PolAd_DonaldTrump_99k4w,https://archive.org/embed/PolAd_DonaldTrump_99k4w,Hillary for America,Multiple,PRES,"2,016.00","Candidate Biography, Foreign Policy, Nuclear, ...",Donald Trump,campaign,mixed,2016-09-12 14:51:51,32.00,Monday,Philadelphia,PA,4.00,8,Clinton,D
2016-08-01 08:48:14,48395,4176,KYW,"Philadelphia, PA",Eyewitness News at 430,news,2016-08-01 08:48:14,2016-08-01 08:48:46,PolAd_DonaldTrump_99k4w,https://archive.org/embed/PolAd_DonaldTrump_99k4w,Hillary for America,Multiple,PRES,"2,016.00","Candidate Biography, Foreign Policy, Nuclear, ...",Donald Trump,campaign,mixed,2016-09-12 14:51:51,32.00,Monday,Philadelphia,PA,4.00,8,Clinton,D
2016-08-01 09:09:45,378441,7459,MSNBCW,"San Francisco-Oakland-San Jose, CA",First Look,news,2016-08-01 09:09:45,2016-08-01 09:10:15,PolAd_HillaryClinton_4gvic,https://archive.org/embed/PolAd_HillaryClinton...,Make America Number One,Super PAC,PRES,"2,016.00",Candidate Biography,Hillary Clinton,campaign,con,2016-10-21 10:40:01,30.00,Monday,San Francisco-Oakland-San Jose,National,1.00,9,R_PAC,R


In [98]:
all_df.shape, cand_date_df.shape

((364718, 19), (83349, 27))

## Pickling Dataframe - Sorted with Sponsors and Party Columns Added

In [99]:
# commenting out to prevent over-write -- it is in the data folder

# cand_date_df.to_pickle('data/pres_sorted_with_sponsors_and_party.pickle')

## EDA | Cleaning | Feature Engineering Summary

- limited dataset to presidential election and excluded dates outside of 8/1/16 - 11/8/16

### Created 8 new features:

- `ad_length`
- `day_of_week`
- `metro_area`
- `state`
- `subject_count`
- `hour_aired`
- `sponsor_class`
- `party`

Created another feature `time_from_creation`, but later dropped it because it did not add value

### Further Cleaning / Feature Engineering in other Project 2 notebooks:

- `data/pres_sorted_with_sponsors_and_party.pickle` is our primary cleaned dataset -- used as basis for `State_Daily_Ad_Counts_7DMA` and `State_Daily_Ad_Mix_by_Party_7DMA` time series analysis notebooks -- only further modified slightly in those notebook (removed NY observations -- see bullet point below)


- Filtered out NY observations in `State_Daily_Ad_Counts_7DMA` pickled file as `data/political_tv_ads_cleaned_for_visuals.pickle` -- this is the basis for `Political_TV_Ads_Visualizations_April_21` notebook


- created a counting dictionary of subjects to see which subjects were mentioned the most - see `subject_df` in `Political_TV_Ads_Visualizations_April_21` notebook


- Time series data notebooks - `State_Daily_Ad_Counts_7DMA` and `State_Daily_Ad_Mix_by_Party_7DMA`


- `State_Time_Series_Visualizations_April_21` makes use of pickled files from the Time Series data notebooks to generate visualizations (`data/state_party_ad_counts_7DMA.pickle` and `data/state_party_neg_ad_mix_7DMA.pickle`, respectively) -- also makes use of state poll data (`data/rcp_state_polls_amended.csv`)